In [1]:
%config IPCompleter.greedy=True

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pandas as pd

filename = './de_ch_vocabulary_spacy.csv'

word = 'word'
base = 'base'
names = [word, base]

voc = pd.read_csv(filename, sep=',', encoding='ansi', comment='#', names=names)
len(voc)

355354

### Import Words from Ionesoft with grammar errors
Ich nehme an ein paar davon haben doppel s anstatt ß und wurden darum nicht gefunden.

In [6]:
import pickle

missing_words = pickle.load(open('missing_words_grammar.p', 'rb'))
missing_words[:5]

['apgemeldet',
 'aplikation',
 'gruess',
 'allgemeinbildenden-unterricht',
 'remoteddesktop']

#### Rechtschreibefehler Lösungsversuch

In [23]:
import itertools
import Levenshtein

def similarity(w1, w2):
    return Levenshtein.jaro(w1.lower(), w2.lower())

def get_closest_match(wrong_word):
    voc['sim'] = voc[word].apply(lambda w: similarity(str(w), wrong_word))
    return voc.iloc[voc['sim'].idxmax()]['base']

In [25]:
get_closest_match('apgemeldet')

'abmelden'

In [19]:
error_correct_dict = dict.fromkeys(missing_words)

In [26]:
for w in missing_words:
    error_correct_dict[w] = get_closest_match(w)

In [29]:
import csv

with open('ionesoft_grammar_corrected.csv', 'w') as f:
    for key in error_correct_dict.keys():
        f.write("%s,%s\n"%(key, error_correct_dict[key]))